![Renode](https://antmicro.com/OpenSource/assets/images/projects/renode.png)

## Install requirements

In [ ]:
!pip install -q git+https://github.com/antmicro/pyrenode.git git+https://github.com/antmicro/renode-colab-tools.git
!mkdir -p renode && cd renode && wget http://dl.antmicro.com/projects/renode/builds/renode-1.11+colab.linux-portable.tar.gz &&  tar -xzf renode-1.11+colab.linux-portable.tar.gz --strip 1
!pip install -q -r renode/tests/requirements.txt
!git clone --quiet https://github.com/antmicro/tensorflow-arduino-examples.git

import os
from renode_colab_tools import *
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']

In [ ]:
!curl -fsSL https://raw.githubusercontent.com/arduino/arduino-cli/master/install.sh | sh
!bin/arduino-cli core install arduino:mbed
!bin/arduino-cli lib install Arduino_LSM9DS1@1.1.0
!git clone --quiet https://github.com/antmicro/tensorflow-arduino-examples.git && cp -r tensorflow-arduino-examples/tensorflow $HOME/Arduino/libraries
!sed -i'' '/#define DEBUG_SERIAL_OBJECT/s/(Serial)/(Serial1)/' $HOME/Arduino/libraries/tensorflow/src/tensorflow/lite/micro/arduino/debug_log.cpp
!bin/arduino-cli compile -b arduino:mbed:nano33ble --output-dir binaries/magic_wand $HOME/Arduino/libraries/tensorflow/examples/magic_wand

## Run a magic-wand example in Renode

In [ ]:
import time
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)
tell_renode('using sysbus')
tell_renode('mach create')
tell_renode('machine LoadPlatformDescription @/content/renode/platforms/boards/arduino_nano_33_ble.repl')
tell_renode('sysbus LoadELF @/content/binaries/magic_wand/magic_wand.ino.elf')

tell_renode('uart0 CreateFileBackend @uart.dump true')
tell_renode('logLevel 3')
tell_renode('machine EnableProfiler "metrics.dump"')
tell_renode('sysbus.twi0.lsm9ds1_imu FeedAccelerationSample @/content/tensorflow-arduino-examples/examples/magic-wand/angle_rotated.data')
tell_renode('s')
time.sleep(5) #waits for creating uart.dump
!timeout 60 tail -c+2 -f renode/uart.dump | sed '/\* \* \* \* \* \* \* \*/ q'
tell_renode('q')
expect_cli('Renode is quitting')
shutdown_renode()

## Renode metrics analysis

In [ ]:
from renode.tools.metrics_analyzer.metrics_parser import MetricsParser
init_notebook_mode(connected=False)
parser = MetricsParser('renode/metrics.dump')

In [ ]:
configure_plotly_browser_state()
show_executed_instructions(parser)

In [ ]:
configure_plotly_browser_state()
show_memory_access(parser)

In [ ]:
configure_plotly_browser_state()
show_exceptions(parser)

In [ ]:
configure_plotly_browser_state()
show_peripheral_access(parser)